In [24]:
import datetime
import sys
import os

import IPython
import IPython.display
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from cond_rnn import ConditionalRNN
from sklearn.preprocessing import MinMaxScaler
from pmdarima.arima import auto_arima

import src.preprocessing
from src.functions import load_data, mape, TimeSeriesTensor, create_evaluation_df, series_to_supervised, plot_train_history, validation

np.set_printoptions(threshold=sys.maxsize)

In [25]:
GEP1 = pd.read_csv('../data/GEP/Consumption_15min.csv', index_col=0, header=0, names=['value'])
GEP2 = pd.read_csv('../data/GEP/B2_Consumption_15min.csv', index_col=0, header=0, names=['value'])
GEP4 = pd.read_csv('../data/GEP/B4_Consumption_15min.csv', index_col=0, header=0, names=['value'])
datasets = [GEP1, GEP2, GEP4]
names = ['GEP1', 'GEP2', 'GEP4']

In [10]:
class Baseline(tf.keras.Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs
        result = inputs[:, :, self.label_index]
        return result[:, :, tf.newaxis]

In [14]:
metrics = pd.DataFrame(columns=['mape','rmse', 'B'], index=range(len(datasets)))
from progressbar import ProgressBar
pbar = ProgressBar()
for i,df in enumerate(datasets):
    train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df)
    baseline = Baseline(label_index = 0)
    baseline.compile(loss=tf.losses.MeanSquaredError(), metrics=[tf.metrics.MeanSquaredError()])
    predictions_B = baseline.predict(test_inputs['X'])
    eval_df_B = create_evaluation_df(predictions_B.reshape(-1,24), test_inputs, 24, y_scaler)
    mape = validation(eval_df_B['prediction'], eval_df_B['actual'], 'MAE')
    #print('mape {}'.format(mape))
    rmse = validation(eval_df_B['prediction'], eval_df_B['actual'], 'RMSE')
    #print('rmse {}'.format(rmse))
    metrics.loc[i] = pd.Series({'mape':mape, 'rmse':rmse, 'B':names[i]})
metrics.to_csv('./results/GEP/baseline.csv')

In [21]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24

FULL_LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(24, 15)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])

metrics = pd.DataFrame(columns=['mae','rmse', 'B'], index=range(len(datasets)))
for i,df in enumerate(datasets):
    train_longer_inputs, test_longer_inputs, y_scaler = MIMO_fulldata_preparation(df)
    # full data LSTM MIMO compilation and fit
    FULL_LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    for j in range(1,11):
        print('RUN:{}'.format(j))
        history = FULL_LSTMIMO.fit(train_longer_inputs['X'], train_longer_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

        FD_predictions = FULL_LSTMIMO.predict(test_longer_inputs['X'])
        FD_eval_df = create_evaluation_df(FD_predictions, test_longer_inputs, HORIZON, y_scaler)
        mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
        rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
        #print('rmse {}'.format(rmse))
        metrics.loc[10*i + j] = pd.Series({'mae':mae, 'rmse':rmse, 'B': names[i]})
metrics.to_csv('./results/GEP/fulldata10_LSTM.csv')

RUN:1
Epoch 1/100
112/112 [==============================] - 2s 21ms/step - loss: 0.0502 - mean_squared_error: 0.0502 - val_loss: 0.0429 - val_mean_squared_error: 0.0429
Epoch 2/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0244 - mean_squared_error: 0.0244 - val_loss: 0.0213 - val_mean_squared_error: 0.0213
Epoch 3/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0169 - mean_squared_error: 0.0169 - val_loss: 0.0266 - val_mean_squared_error: 0.0266
Epoch 4/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.0312 - val_mean_squared_error: 0.0312
Epoch 5/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0118 - mean_squared_error: 0.0118 - val_loss: 0.0295 - val_mean_squared_error: 0.0295
Epoch 6/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0105 - mean_squared_error: 0.0105 - val_loss: 0.0377 - val_mean_squared_error: 0.0377
Epoch 7/10

Epoch 39/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0026 - mean_squared_error: 0.0026 - val_loss: 0.0302 - val_mean_squared_error: 0.0302
Epoch 40/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0025 - mean_squared_error: 0.0025 - val_loss: 0.0306 - val_mean_squared_error: 0.0306
Epoch 41/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0297 - val_mean_squared_error: 0.0297
Epoch 42/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0025 - mean_squared_error: 0.0025 - val_loss: 0.0306 - val_mean_squared_error: 0.0306
Epoch 43/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0300 - val_mean_squared_error: 0.0300
Epoch 44/100
112/112 [==============================] - 1s 13ms/step - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0297 - val_mean_squared_error: 0.0297
Epoch 45/1

RUN:5
Epoch 1/100
112/112 [==============================] - 1s 13ms/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0284 - val_mean_squared_error: 0.0284
Epoch 2/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0292 - val_mean_squared_error: 0.0292
Epoch 3/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0017 - mean_squared_error: 0.0017 - val_loss: 0.0283 - val_mean_squared_error: 0.0283
Epoch 4/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0279 - val_mean_squared_error: 0.0279
Epoch 5/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0017 - mean_squared_error: 0.0017 - val_loss: 0.0275 - val_mean_squared_error: 0.0275
Epoch 6/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0275 - val_mean_squared_error: 0.0275
Epoch 7/10

Epoch 8/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0292 - val_mean_squared_error: 0.0292
Epoch 9/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0293 - val_mean_squared_error: 0.0293
Epoch 10/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0285 - val_mean_squared_error: 0.0285
Epoch 11/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0295 - val_mean_squared_error: 0.0295
Epoch 12/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0298 - val_mean_squared_error: 0.0298
Epoch 13/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0296 - val_mean_squared_error: 0.0296
Epoch 14/100

Epoch 10/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0029 - mean_squared_error: 0.0029 - val_loss: 0.0165 - val_mean_squared_error: 0.0165
Epoch 11/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0029 - mean_squared_error: 0.0029 - val_loss: 0.0170 - val_mean_squared_error: 0.0170
Epoch 12/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0028 - mean_squared_error: 0.0028 - val_loss: 0.0173 - val_mean_squared_error: 0.0173
Epoch 13/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0027 - mean_squared_error: 0.0027 - val_loss: 0.0166 - val_mean_squared_error: 0.0166
Epoch 14/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0026 - mean_squared_error: 0.0026 - val_loss: 0.0159 - val_mean_squared_error: 0.0159
Epoch 15/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0026 - mean_squared_error: 0.0026 - val_loss: 0.0174 - val_mean_squared_error: 0.0174
Epoch 16/1

...

Epoch 25/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0247 - val_mean_squared_error: 0.0247
Epoch 26/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0245 - val_mean_squared_error: 0.0245
Epoch 27/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0251 - val_mean_squared_error: 0.0251
RUN:10
Epoch 1/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0241 - val_mean_squared_error: 0.0241
Epoch 2/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0254 - val_mean_squared_error: 0.0254
Epoch 3/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0264 - val_mean_squared_error: 0.0264
Epoch 

Epoch 21/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0307 - val_mean_squared_error: 0.0307
Epoch 22/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0302 - val_mean_squared_error: 0.0302
Epoch 23/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0300 - val_mean_squared_error: 0.0300
Epoch 24/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0298 - val_mean_squared_error: 0.0298
Epoch 25/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0299 - val_mean_squared_error: 0.0299
Epoch 26/100
112/112 [==============================] - 1s 13ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0298 - val_mean_squared_error: 0.0298
Epoch 27/1

Epoch 5/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0299 - val_mean_squared_error: 0.0299
Epoch 6/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0293 - val_mean_squared_error: 0.0293
Epoch 7/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0300 - val_mean_squared_error: 0.0300
Epoch 8/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0296 - val_mean_squared_error: 0.0296
Epoch 9/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0302 - val_mean_squared_error: 0.0302
Epoch 10/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0293 - val_mean_squared_error: 0.0293
Epoch 11/100
11

Epoch 24/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0010 - mean_squared_error: 0.0010 - val_loss: 0.0291 - val_mean_squared_error: 0.0291
RUN:6
Epoch 1/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0010 - mean_squared_error: 0.0010 - val_loss: 0.0302 - val_mean_squared_error: 0.0302
Epoch 2/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0010 - mean_squared_error: 0.0010 - val_loss: 0.0296 - val_mean_squared_error: 0.0296
Epoch 3/100
112/112 [==============================] - 1s 13ms/step - loss: 0.0010 - mean_squared_error: 0.0010 - val_loss: 0.0289 - val_mean_squared_error: 0.0289
Epoch 4/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0294 - val_mean_squared_error: 0.0294
Epoch 5/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0291 - val_mean_squared_error: 0.0291
Epoch 6/1

112/112 [==============================] - 2s 14ms/step - loss: 9.8553e-04 - mean_squared_error: 9.8553e-04 - val_loss: 0.0296 - val_mean_squared_error: 0.0296
Epoch 9/100
112/112 [==============================] - 2s 14ms/step - loss: 9.8546e-04 - mean_squared_error: 9.8546e-04 - val_loss: 0.0296 - val_mean_squared_error: 0.0296
Epoch 10/100
112/112 [==============================] - 2s 14ms/step - loss: 9.8305e-04 - mean_squared_error: 9.8305e-04 - val_loss: 0.0301 - val_mean_squared_error: 0.0301
Epoch 11/100
112/112 [==============================] - 2s 14ms/step - loss: 9.8796e-04 - mean_squared_error: 9.8796e-04 - val_loss: 0.0297 - val_mean_squared_error: 0.0297
Epoch 12/100
112/112 [==============================] - 2s 14ms/step - loss: 9.9715e-04 - mean_squared_error: 9.9715e-04 - val_loss: 0.0301 - val_mean_squared_error: 0.0301
Epoch 13/100
112/112 [==============================] - 2s 15ms/step - loss: 9.8323e-04 - mean_squared_error: 9.8323e-04 - val_loss: 0.0297 - val_mea

In [19]:
NUM_CELLS = 32
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24
# ConditionalRNN with the functional API.
#i = Input(shape=[HORIZON, 5], name='time-series')
#c1 = Input(shape=[8], name='one-hot')
#c2 = Input(shape=[2], name='lags-as-features')
# add the condition tensor here.
#x = ConditionalRNN(NUM_CELLS, cell='LSTM', name='cond_rnn_0')([i, c1, c2])
# Dense layer to output
#x = Dense(HORIZON)(x)
#ConditionalRNN = Model(inputs=[i, c1, c2], outputs=[x])

ConditionalRNN = Sequential(layers=[ConditionalRNN(32, cell='LSTM'),
                                    Dense(HORIZON)])

metrics = pd.DataFrame(columns=['mae','rmse', 'B'], index=range(len(datasets)))
from progressbar import ProgressBar
pbar = ProgressBar()
for i,df in enumerate(datasets):
    train_inputs, test_inputs, train_c1, test_c1, train_c2, test_c2, y_scaler = ConditionalRNN_data_preparation(df)
    # full data LSTM MIMO compilation and fit
    ConditionalRNN.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    for j in range(1,11):
        history = ConditionalRNN.fit([train_inputs['X'],train_c1,train_c2], train_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

        #save_model(ConditionalRNN, i)
        predictions = ConditionalRNN.predict([test_inputs['X'],test_c1,test_c2])
        eval_df = create_evaluation_df(predictions, test_inputs, HORIZON, y_scaler)
        mae = validation(eval_df['prediction'], eval_df['actual'], 'MAE')
        rmse = validation(eval_df['prediction'], eval_df['actual'], 'RMSE')
    #print('rmse {}'.format(rmse))
        metrics.loc[10*i + j] = pd.Series({'mae':mae, 'rmse':rmse, 'B': names[i]})
metrics.to_csv('./results/GEP/ConditionalRNN10_sequential.csv')

Epoch 1/100
Consider rewriting this model with the Functional API.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by pass

112/112 [==============================] - 2s 16ms/step - loss: 0.0063 - mean_squared_error: 0.0063 - val_loss: 0.0398 - val_mean_squared_error: 0.0398
Epoch 4/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0064 - mean_squared_error: 0.0064 - val_loss: 0.0387 - val_mean_squared_error: 0.0387
Epoch 5/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0062 - mean_squared_error: 0.0062 - val_loss: 0.0475 - val_mean_squared_error: 0.0475
Epoch 6/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0060 - mean_squared_error: 0.0060 - val_loss: 0.0431 - val_mean_squared_error: 0.0431
Epoch 7/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0058 - mean_squared_error: 0.0058 - val_loss: 0.0439 - val_mean_squared_error: 0.0439
Epoch 8/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0058 - mean_squared_error: 0.0058 - val_loss: 0.0424 - val_mean_squared_error: 0.0424
Epoch 9/100
112/112 [=======

112/112 [==============================] - 2s 15ms/step - loss: 0.0042 - mean_squared_error: 0.0042 - val_loss: 0.0443 - val_mean_squared_error: 0.0443
Epoch 3/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0043 - mean_squared_error: 0.0043 - val_loss: 0.0419 - val_mean_squared_error: 0.0419
Epoch 4/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0413 - val_mean_squared_error: 0.0413
Epoch 5/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0042 - mean_squared_error: 0.0042 - val_loss: 0.0430 - val_mean_squared_error: 0.0430
Epoch 6/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0435 - val_mean_squared_error: 0.0435
Epoch 7/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0039 - mean_squared_error: 0.0039 - val_loss: 0.0454 - val_mean_squared_error: 0.0454
Epoch 8/100
112/112 [=======

Epoch 7/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0442 - val_mean_squared_error: 0.0442
Epoch 8/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0031 - mean_squared_error: 0.0031 - val_loss: 0.0376 - val_mean_squared_error: 0.0376
Epoch 9/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0029 - mean_squared_error: 0.0029 - val_loss: 0.0402 - val_mean_squared_error: 0.0402
Epoch 10/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0032 - mean_squared_error: 0.0032 - val_loss: 0.0428 - val_mean_squared_error: 0.0428
Epoch 11/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0409 - val_mean_squared_error: 0.0409
Epoch 12/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0032 - mean_squared_error: 0.0032 - val_loss: 0.0422 - val_mean_squared_error: 0.0422
Epoch 13/100


112/112 [==============================] - 2s 17ms/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0432 - val_mean_squared_error: 0.0432
Epoch 2/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0043 - mean_squared_error: 0.0043 - val_loss: 0.0476 - val_mean_squared_error: 0.0476
Epoch 3/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0453 - val_mean_squared_error: 0.0453
Epoch 4/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.0425 - val_mean_squared_error: 0.0425
Epoch 5/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0477 - val_mean_squared_error: 0.0477
Epoch 6/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0464 - val_mean_squared_error: 0.0464
Epoch 7/100
112/112 [=======

...

Epoch 2/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0021 - mean_squared_error: 0.0021 - val_loss: 0.0432 - val_mean_squared_error: 0.0432
Epoch 3/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0021 - mean_squared_error: 0.0021 - val_loss: 0.0400 - val_mean_squared_error: 0.0400
Epoch 4/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0415 - val_mean_squared_error: 0.0415
Epoch 5/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0021 - mean_squared_error: 0.0021 - val_loss: 0.0419 - val_mean_squared_error: 0.0419
Epoch 6/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0021 - mean_squared_error: 0.0021 - val_loss: 0.0407 - val_mean_squared_error: 0.0407
Epoch 7/100
112/112 [==============================] - 2s 13ms/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0376 - val_mean_squared_error: 0.0376
Epoch 8/100
112/

Epoch 16/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0341 - val_mean_squared_error: 0.0341
Epoch 17/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0414 - val_mean_squared_error: 0.0414
Epoch 18/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0407 - val_mean_squared_error: 0.0407
Epoch 19/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0410 - val_mean_squared_error: 0.0410
Epoch 20/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0407 - val_mean_squared_error: 0.0407
Epoch 21/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0416 - val_mean_squared_error: 0.0416
Epoch 22/1

112/112 [==============================] - 2s 17ms/step - loss: 0.0090 - mean_squared_error: 0.0090 - val_loss: 0.0263 - val_mean_squared_error: 0.0263
Epoch 7/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0253 - val_mean_squared_error: 0.0253
Epoch 8/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0077 - mean_squared_error: 0.0077 - val_loss: 0.0262 - val_mean_squared_error: 0.0262
Epoch 9/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0070 - mean_squared_error: 0.0070 - val_loss: 0.0286 - val_mean_squared_error: 0.0286
Epoch 10/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0063 - mean_squared_error: 0.0063 - val_loss: 0.0264 - val_mean_squared_error: 0.0264
Epoch 11/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.0271 - val_mean_squared_error: 0.0271
Consider rewriting this mo

Epoch 1/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0272 - val_mean_squared_error: 0.0272
Epoch 2/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0273 - val_mean_squared_error: 0.0273
Epoch 3/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0277 - val_mean_squared_error: 0.0277
Epoch 4/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0268 - val_mean_squared_error: 0.0268
Epoch 5/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0025 - mean_squared_error: 0.0025 - val_loss: 0.0274 - val_mean_squared_error: 0.0274
Epoch 6/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0275 - val_mean_squared_error: 0.0275
Epoch 7/100
112/

Epoch 3/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0309 - val_mean_squared_error: 0.0309
Epoch 4/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0300 - val_mean_squared_error: 0.0300
Epoch 5/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0312 - val_mean_squared_error: 0.0312
Epoch 6/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0288 - val_mean_squared_error: 0.0288
Epoch 7/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0295 - val_mean_squared_error: 0.0295
Epoch 8/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0300 - val_mean_squared_error: 0.0300
Epoch 9/100
112/

In [66]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24

FULL_LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(24, 16)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])

metrics = pd.DataFrame(columns=['mae','rmse', 'B'], index=range(len(datasets)))
for i,df in enumerate(datasets[:-1]):
    train_longer_inputs, test_longer_inputs, y_scaler = MIMO_fulldataT_preparation(df)
    # full data LSTM MIMO compilation and fit
    FULL_LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    for j in range(1,11):
        print('RUN:{}'.format(j))
        history = FULL_LSTMIMO.fit(train_longer_inputs['X'], train_longer_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

        FD_predictions = FULL_LSTMIMO.predict(test_longer_inputs['X'])
        FD_eval_df = create_evaluation_df(FD_predictions, test_longer_inputs, HORIZON, y_scaler)
        mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
        rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
        #print('rmse {}'.format(rmse))
        metrics.loc[10*i + j] = pd.Series({'mae':mae, 'rmse':rmse, 'B': names[i]})
metrics.to_csv('./results/GEP/Temp_LSTM.csv')

RUN:1
Epoch 1/100
112/112 [==============================] - 2s 22ms/step - loss: 0.0528 - mean_squared_error: 0.0528 - val_loss: 0.0453 - val_mean_squared_error: 0.0453
Epoch 2/100
112/112 [==============================] - 2s 21ms/step - loss: 0.0265 - mean_squared_error: 0.0265 - val_loss: 0.0261 - val_mean_squared_error: 0.0261
Epoch 3/100
112/112 [==============================] - 2s 19ms/step - loss: 0.0183 - mean_squared_error: 0.0183 - val_loss: 0.0299 - val_mean_squared_error: 0.0299
Epoch 4/100
112/112 [==============================] - 2s 18ms/step - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.0296 - val_mean_squared_error: 0.0296
Epoch 5/100
112/112 [==============================] - 2s 18ms/step - loss: 0.0119 - mean_squared_error: 0.0119 - val_loss: 0.0352 - val_mean_squared_error: 0.0352
Epoch 6/100
112/112 [==============================] - 2s 18ms/step - loss: 0.0104 - mean_squared_error: 0.0104 - val_loss: 0.0363 - val_mean_squared_error: 0.0363
Epoch 7/10

Epoch 23/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0421 - val_mean_squared_error: 0.0421
Epoch 24/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0029 - mean_squared_error: 0.0029 - val_loss: 0.0412 - val_mean_squared_error: 0.0412
Epoch 25/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0422 - val_mean_squared_error: 0.0422
Epoch 26/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0026 - mean_squared_error: 0.0026 - val_loss: 0.0423 - val_mean_squared_error: 0.0423
Epoch 27/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0446 - val_mean_squared_error: 0.0446
Epoch 28/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0025 - mean_squared_error: 0.0025 - val_loss: 0.0426 - val_mean_squared_error: 0.0426
Epoch 29/1

Epoch 12/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0444 - val_mean_squared_error: 0.0444
Epoch 13/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0457 - val_mean_squared_error: 0.0457
Epoch 14/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0452 - val_mean_squared_error: 0.0452
Epoch 15/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0422 - val_mean_squared_error: 0.0422
Epoch 16/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0447 - val_mean_squared_error: 0.0447
Epoch 17/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0467 - val_mean_squared_error: 0.0467
Epoch 18/1

Epoch 15/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0407 - val_mean_squared_error: 0.0407
Epoch 16/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0017 - mean_squared_error: 0.0017 - val_loss: 0.0387 - val_mean_squared_error: 0.0387
Epoch 17/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0427 - val_mean_squared_error: 0.0427
Epoch 18/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0421 - val_mean_squared_error: 0.0421
Epoch 19/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0417 - val_mean_squared_error: 0.0417
Epoch 20/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0431 - val_mean_squared_error: 0.0431
Epoch 21/1

Epoch 15/100
112/112 [==============================] - 2s 18ms/step - loss: 0.0021 - mean_squared_error: 0.0021 - val_loss: 0.0762 - val_mean_squared_error: 0.0762
Epoch 16/100
112/112 [==============================] - 2s 18ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0743 - val_mean_squared_error: 0.0743
Epoch 17/100
112/112 [==============================] - 2s 18ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0743 - val_mean_squared_error: 0.0743
RUN:3
Epoch 1/100
112/112 [==============================] - 2s 18ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0724 - val_mean_squared_error: 0.0724
Epoch 2/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0731 - val_mean_squared_error: 0.0731
Epoch 3/100
112/112 [==============================] - 2s 18ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0706 - val_mean_squared_error: 0.0706
Epoch 4

Epoch 3/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0017 - mean_squared_error: 0.0017 - val_loss: 0.0700 - val_mean_squared_error: 0.0700
Epoch 4/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0716 - val_mean_squared_error: 0.0716
Epoch 5/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0712 - val_mean_squared_error: 0.0712
Epoch 6/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0713 - val_mean_squared_error: 0.0713
Epoch 7/100
112/112 [==============================] - 2s 15ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0719 - val_mean_squared_error: 0.0719
Epoch 8/100
112/112 [==============================] - 2s 14ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0695 - val_mean_squared_error: 0.0695
Epoch 9/100
112/

Epoch 7/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0750 - val_mean_squared_error: 0.0750
Epoch 8/100
112/112 [==============================] - 2s 16ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0750 - val_mean_squared_error: 0.0750
Epoch 9/100
112/112 [==============================] - 2s 13ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0738 - val_mean_squared_error: 0.0738
Epoch 10/100
112/112 [==============================] - 1s 13ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0796 - val_mean_squared_error: 0.0796
Epoch 11/100
112/112 [==============================] - 1s 13ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0727 - val_mean_squared_error: 0.0727
Epoch 12/100
112/112 [==============================] - 2s 17ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0763 - val_mean_squared_error: 0.0763
Epoch 13/100


In [64]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24

LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(24, 5)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])

metrics = pd.DataFrame(columns=['mae','rmse', 'B'], index=range(len(datasets)))
from progressbar import ProgressBar
pbar = ProgressBar()
for i,df in enumerate(datasets):
    train_inputs, test_inputs, _, _, _, _, y_scaler = ConditionalRNN_data_preparation(df)
    # full data LSTM MIMO compilation and fit
    LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    
    for j in range(1,11):
        history = LSTMIMO.fit(train_inputs['X'], train_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

        FD_predictions = LSTMIMO.predict(test_inputs['X'])
        FD_eval_df = create_evaluation_df(FD_predictions, test_inputs, HORIZON, y_scaler)
        mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
        rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
        #print('rmse {}'.format(rmse))
        metrics.loc[10*i + j] = pd.Series({'mae':mae, 'rmse':rmse, 'B': names[i]})
metrics.to_csv('./results/GEP/LSTM_MIMO10.csv')

Epoch 1/100
112/112 [==============================] - ETA: 0s - loss: 0.0527 - mean_squared_error: 0.0527

KeyboardInterrupt: 

In [26]:
def train_test_split(df, n_test):
    test_df = df.copy()[-n_test:]
    train_df = df.copy()[:len(test_df)]
    return train_df, test_df

In [27]:
def ConditionalRNN_data_preparation(df, n_test=4380, T=24, HORIZON=24):
    df = src.preprocessing.preprocess(df, 'Canada')
    c2 = series_to_supervised(df)
    c1 = df.iloc[:,1:9]
    c1 = c1.loc[c2.index]
    short_df = df.iloc[:,[0,1,-4,-3,-2,-1]].copy()
    #check how to formulate more correctly
    short_df = short_df[146:]
    train_df, test_df = train_test_split(short_df, n_test)
    train_c1, test_c1 = train_test_split(c1, n_test)
    train_c2, test_c2 = train_test_split(c2, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    train_c2[['value(t-24)']] = y_scaler.fit_transform(train_c2[['value(t-24)']])
    train_c2[['value(t-168)']] = y_scaler.fit_transform(train_c2[['value(t-168)']])
    X_scaler = MinMaxScaler()
    train_df[train_df.columns] = X_scaler.fit_transform(train_df)
    test_df[train_df.columns] = X_scaler.fit_transform(test_df)
    tensor_structure = {'X':(range(-T+1, 1), ['value','fractional hour_sin','fractional hour_cos','day of year_sin','day of year_cos'])}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    train_c1 = c1.reindex(train_inputs.dataframe.index)
    train_c2 = c2.reindex(train_inputs.dataframe.index)
    test_c1 = c1.reindex(test_inputs.dataframe.index)
    test_c2 = c2.reindex(test_inputs.dataframe.index)
    return train_inputs, test_inputs, train_c1, test_c1, train_c2, test_c2, y_scaler

In [28]:
def MIMO_fulldata_preparation(df, n_test=4380, T=24, HORIZON=24):
    df = src.preprocessing.preprocess(df, 'Canada')
    df = df.merge(series_to_supervised(df), how='right', left_index=True, right_index=True)
    train_df, test_df = train_test_split(df, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    long_scaler = MinMaxScaler()
    test_df[test_df.columns] = long_scaler.fit_transform(test_df)
    train_df[train_df.columns] = long_scaler.fit_transform(train_df)
    tensor_structure = {'X':(range(-T+1, 1), train_df.columns)}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    return train_inputs, test_inputs, y_scaler

In [40]:
def MIMO_fulldataT_preparation(df, n_test=4380, T=24, HORIZON=24):
    df = src.preprocessing.preprocess(df, 'Canada')
    df = df.merge(series_to_supervised(df), how='right', left_index=True, right_index=True)
    df = df.merge(temp, how='left', left_index=True, right_index=True)
    train_df, test_df = train_test_split(df, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    long_scaler = MinMaxScaler()
    test_df[test_df.columns] = long_scaler.fit_transform(test_df)
    train_df[train_df.columns] = long_scaler.fit_transform(train_df)
    tensor_structure = {'X':(range(-T+1, 1), train_df.columns)}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    return train_inputs, test_inputs, y_scaler

In [38]:
temp = pd.read_csv('../data/GEP/Temp_15min.csv', index_col=0, header=0, names=['temp'])

In [63]:
train_longer_inputs, test_longer_inputs, y_scaler = MIMO_fulldataT_preparation(GEP4)

/home/ubuntu/anaconda3/envs/evgeny/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/ubuntu/anaconda3/envs/evgeny/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [59]:
train_longer_inputs.dataframe[train_longer_inputs.dataframe.isna().any(axis=1)]

Empty DataFrame
Columns: [(target, y, t+1), (target, y, t+2), (target, y, t+3), (target, y, t+4), (target, y, t+5), (target, y, t+6), (target, y, t+7), (target, y, t+8), (target, y, t+9), (target, y, t+10), (target, y, t+11), (target, y, t+12), (target, y, t+13), (target, y, t+14), (target, y, t+15), (target, y, t+16), (target, y, t+17), (target, y, t+18), (target, y, t+19), (target, y, t+20), (target, y, t+21), (target, y, t+22), (target, y, t+23), (target, y, t+24), (X, value, t-23), (X, value, t-22), (X, value, t-21), (X, value, t-20), (X, value, t-19), (X, value, t-18), (X, value, t-17), (X, value, t-16), (X, value, t-15), (X, value, t-14), (X, value, t-13), (X, value, t-12), (X, value, t-11), (X, value, t-10), (X, value, t-9), (X, value, t-8), (X, value, t-7), (X, value, t-6), (X, value, t-5), (X, value, t-4), (X, value, t-3), (X, value, t-2), (X, value, t-1), (X, value, t), (X, working day, t-23), (X, working day, t-22), (X, working day, t-21), (X, working day, t-20), (X, working day, t-19), (X, working day, t-18), (X, working day, t-17), (X, working day, t-16), (X, working day, t-15), (X, working day, t-14), (X, working day, t-13), (X, working day, t-12), (X, working day, t-11), (X, working day, t-10), (X, working day, t-9), (X, working day, t-8), (X, working day, t-7), (X, working day, t-6), (X, working day, t-5), (X, working day, t-4), (X, working day, t-3), (X, working day, t-2), (X, working day, t-1), (X, working day, t), (X, week_1, t-23), (X, week_1, t-22), (X, week_1, t-21), (X, week_1, t-20), (X, week_1, t-19), (X, week_1, t-18), (X, week_1, t-17), (X, week_1, t-16), (X, week_1, t-15), (X, week_1, t-14), (X, week_1, t-13), (X, week_1, t-12), (X, week_1, t-11), (X, week_1, t-10), (X, week_1, t-9), (X, week_1, t-8), (X, week_1, t-7), (X, week_1, t-6), (X, week_1, t-5), (X, week_1, t-4), (X, week_1, t-3), (X, week_1, t-2), (X, week_1, t-1), (X, week_1, t), (X, week_2, t-23), (X, week_2, t-22), (X, week_2, t-21), (X, week_2, t-20), ...]
Index: []

[0 rows x 408 columns]

In [62]:
GEP4[GEP4.isna().any(axis=1)]

,value
